In [1]:
import torch, torchvision
from torchvision import datasets
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

In [2]:
print(torch.__version__)
print(torchvision.__version__)

2.7.1+cpu
0.22.1+cpu


In [3]:
train_ds = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_ds = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [4]:
BATCH_SIZE = 64

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

print(len(test_loader))
for x, y in test_loader:
    print(f"X's shape [N, C, H, W]: {x.shape}, datatype: {x.dtype}")
    print(f"Y's shape: {y.shape}, datatype: {y.dtype}")
    break

157
X's shape [N, C, H, W]: torch.Size([64, 1, 28, 28]), datatype: torch.float32
Y's shape: torch.Size([64]), datatype: torch.int64


In [5]:
print(torch.accelerator)

<module 'torch.accelerator' from 'C:\\Users\\virat\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\torch\\accelerator\\__init__.py'>


In [6]:
#device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available else "cpu"
device = "cpu"
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): ReLU()
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)


In [7]:
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=1e-3)

In [8]:
print(len(train_loader.dataset))
print(len(train_loader))
print(len(test_loader))
print(len(test_loader.dataset))

60000
938
157
10000


In [9]:
def train(dataloader, model, loss_fn, optim):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # compute loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # backprop
        loss.backward()
        optim.step()
        optim.zero_grad() # reset gradients to 0

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X) # outputs 10 logits
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size

    print(f"Test Error: \n Accuracy: {100 * correct}%, Avg loss: {test_loss}\n")

In [11]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t + 1}\n" + f"-"*20)
    train(train_loader, model, loss_fn, optim)
    test(test_loader, model, loss_fn)

print("\nOwari da!!!")

Epoch 1
--------------------
loss: 2.294928  [   64/60000]
loss: 0.749734  [ 6464/60000]
loss: 0.450124  [12864/60000]
loss: 0.529639  [19264/60000]
loss: 0.508376  [25664/60000]
loss: 0.449764  [32064/60000]
loss: 0.413610  [38464/60000]
loss: 0.551410  [44864/60000]
loss: 0.480557  [51264/60000]
loss: 0.578094  [57664/60000]
Test Error: 
 Accuracy: 82.37%, Avg loss: 0.46492158028350516

Epoch 2
--------------------
loss: 0.344311  [   64/60000]
loss: 0.419504  [ 6464/60000]
loss: 0.290262  [12864/60000]
loss: 0.386299  [19264/60000]
loss: 0.395116  [25664/60000]
loss: 0.335799  [32064/60000]
loss: 0.323321  [38464/60000]
loss: 0.471869  [44864/60000]
loss: 0.455620  [51264/60000]
loss: 0.455168  [57664/60000]
Test Error: 
 Accuracy: 83.87%, Avg loss: 0.4288744413928621

Epoch 3
--------------------
loss: 0.271965  [   64/60000]
loss: 0.343783  [ 6464/60000]
loss: 0.281959  [12864/60000]
loss: 0.341144  [19264/60000]
loss: 0.357026  [25664/60000]
loss: 0.329469  [32064/60000]
loss: 0.

In [12]:
torch.save(model.state_dict(), "model1.pth")

In [13]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model1.pth", weights_only=True))

<All keys matched successfully>

In [14]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_ds[555][0], test_ds[555][1] # respective image and label

with torch.no_grad():
    x = x.to(device)
    pred = model(x)

    # just scoping out what up
    print(f"Logits' shape: {pred.shape}")
    print(f"Returned logits pre indexing: {pred}")
    print(f"Returned logits post indexing: {pred[0]}")
    # Converting to values
    predicted, actual = classes[pred.argmax(1)], classes[y] # returns max value idx across the row
    #predicted, actual = classes[pred[0].argmax(0)], classes[y] # returns max value idx in the only remaining dimension since we are indexing into pred with [0]

print(f"Actual: {actual}, Predicted: {predicted}")

Logits' shape: torch.Size([1, 10])
Returned logits pre indexing: tensor([[ -4.0576,  18.4834,  -8.7461,  -1.4882,  -0.8848, -18.5769,  -2.6766,
         -18.5907, -16.2656, -10.6099]])
Returned logits post indexing: tensor([ -4.0576,  18.4834,  -8.7461,  -1.4882,  -0.8848, -18.5769,  -2.6766,
        -18.5907, -16.2656, -10.6099])
Actual: Trouser, Predicted: Trouser
